# 共享单车预测 - 回归问题

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## 1. 导入并处理数据

### 1.1. 导入数据

In [2]:
df = pd.read_csv("train.csv") 
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null object
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.6+ KB
None


### 1.2. 处理时间数据

In [3]:
df['hour']=pd.DatetimeIndex(df.datetime).hour
df['day']=pd.DatetimeIndex(df.datetime).dayofweek
df['month']=pd.DatetimeIndex(df.datetime).month

df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,day,month
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,0,5,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1,5,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2,5,1
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,3,5,1
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,4,5,1


In [4]:
df_origin = df
df = df.drop(['datetime','casual','registered'], axis = 1)
df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,day,month
0,1,0,0,1,9.84,14.395,81,0.0,16,0,5,1
1,1,0,0,1,9.02,13.635,80,0.0,40,1,5,1
2,1,0,0,1,9.02,13.635,80,0.0,32,2,5,1
3,1,0,0,1,9.84,14.395,75,0.0,13,3,5,1
4,1,0,0,1,9.84,14.395,75,0.0,1,4,5,1


### 1.3. 提取数据

In [5]:
data = df.values
print(data.shape)
data

(10886, 12)


array([[ 1.,  0.,  0., ...,  0.,  5.,  1.],
       [ 1.,  0.,  0., ...,  1.,  5.,  1.],
       [ 1.,  0.,  0., ...,  2.,  5.,  1.],
       ...,
       [ 4.,  0.,  1., ..., 21.,  2., 12.],
       [ 4.,  0.,  1., ..., 22.,  2., 12.],
       [ 4.,  0.,  1., ..., 23.,  2., 12.]])

In [6]:
print(data[0, -4])
count_idx = -4

16.0


### 1.4. 划分训练集和测试集

In [7]:
# 划分训练集和测试集
n = data.shape[0]

train_start = 0
train_end = int(np.floor(0.8*n))

test_start = train_end + 1
test_end = n

data_train = data[np.arange(train_start, train_end), :] # 训练数据
data_test = data[np.arange(test_start, test_end), :] # 测试数据

### 标准化处理(废弃)

没有最大最小化编码(规范化到[-1,1]之间)效果好

### 1.5. 最大最小化 数据缩放处理

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler_train = MinMaxScaler()
data_train = scaler_train.fit_transform(data_train)
print(data_train)

scaler_test = MinMaxScaler()
data_test = scaler_test.fit_transform(data_test)
print(data_test)

[[0.         0.         0.         ... 0.         0.83333333 0.        ]
 [0.         0.         0.         ... 0.04347826 0.83333333 0.        ]
 [0.         0.         0.         ... 0.08695652 0.83333333 0.        ]
 ...
 [0.66666667 0.         0.         ... 0.08695652 1.         0.63636364]
 [0.66666667 0.         0.         ... 0.13043478 1.         0.63636364]
 [0.66666667 0.         0.         ... 0.17391304 1.         0.63636364]]
[[0.         0.         0.         ... 0.26086957 1.         0.        ]
 [0.         0.         0.         ... 0.30434783 1.         0.        ]
 [0.         0.         0.         ... 0.34782609 1.         0.        ]
 ...
 [1.         0.         1.         ... 0.91304348 0.33333333 1.        ]
 [1.         0.         1.         ... 0.95652174 0.33333333 1.        ]
 [1.         0.         1.         ... 1.         0.33333333 1.        ]]


### 1.6. 提取x, y

In [9]:
# 训练
y_train = data_train[:, count_idx]
y_train.shape = (len(y_train), 1)
x_train = np.delete(data_train, count_idx, axis=1)

# 测试
y_test = data_test[:, count_idx]
y_test.shape = (len(y_test), 1)
x_test = np.delete(data_test, count_idx, axis=1)

## 模型创建

In [10]:
# 根据样本样式定义创建占位符（1列）
x = tf.placeholder(tf.float32,[None,11]) # 11个特征
y = tf.placeholder(tf.float32,[None,1]) # 一个输出

In [11]:
# 参数：(输入数据, 前一个神经元个数, 下一层神经元个数, 激活函数)
def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

In [12]:
# 第一层网络
Z1 = add_layer(x, 11, 100, tf.nn.tanh)
# 第二层网络
prediction = add_layer(Z1, 100, 1, tf.nn.tanh)

In [13]:
# 二次代阶函数 回归问题
loss = tf.reduce_mean(tf.square(y - prediction))

In [14]:
lr = 0.001
# 定义一个梯度下降法来进行训练的优化器
# train =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
train =  tf.train.AdamOptimizer(lr).minimize(loss)

## 模型训练

In [15]:
# 初始化变量  
init = tf.global_variables_initializer()

In [16]:
# 启动会话
sess = tf.Session()
sess.run(init)

In [17]:
for step in range(1000):
    sess.run(train, feed_dict={x:x_train, y:y_train})

In [18]:
# 获取预测值
prediction_value = sess.run(prediction, feed_dict={x: x_test})
for i in range(len(y_test)):
    print('预测值：', prediction_value[i], '真实值：', y_test[i])

预测值： [0.39967486] 真实值： [0.01844262]
预测值： [0.38861987] 真实值： [0.06864754]
预测值： [0.3647841] 真实值： [0.10963115]
预测值： [0.3644673] 真实值： [0.23360656]
预测值： [0.20269014] 真实值： [0.37192623]
预测值： [0.27472386] 真实值： [0.44672131]
预测值： [0.00803952] 真实值： [0.50204918]
预测值： [0.2871652] 真实值： [0.53483607]
预测值： [0.34244502] 真实值： [0.53790984]
预测值： [0.37863857] 真实值： [0.46004098]
预测值： [0.83997566] 真实值： [0.43032787]
预测值： [0.8556211] 真实值： [0.39036885]
预测值： [-0.02370433] 真实值： [0.23360656]
预测值： [0.7439461] 真实值： [0.18852459]
预测值： [0.5857507] 真实值： [0.19672131]
预测值： [0.6768362] 真实值： [0.21209016]
预测值： [0.6402309] 真实值： [0.15983607]
预测值： [0.22922218] 真实值： [0.08811475]
预测值： [-0.1307583] 真实值： [0.03278689]
预测值： [-0.4525815] 真实值： [0.0102459]
预测值： [-0.27656108] 真实值： [0.00409836]
预测值： [-0.22254668] 真实值： [0.00409836]
预测值： [-0.4106762] 真实值： [0.0102459]
预测值： [-0.29165262] 真实值： [0.02459016]
预测值： [-0.33710352] 真实值： [0.14344262]
预测值： [-0.4332219] 真实值： [0.41598361]
预测值： [-0.02032651] 真实值： [0.61885246]
预测值： [-0.4438184] 真实值： [0.281762

预测值： [0.15233697] 真实值： [0.16803279]
预测值： [0.19261082] 真实值： [0.47438525]
预测值： [0.19993229] 真实值： [0.6977459]
预测值： [0.25407678] 真实值： [0.3442623]
预测值： [0.29685786] 真实值： [0.20286885]
预测值： [0.4497869] 真实值： [0.27971311]
预测值： [0.3933447] 真实值： [0.35040984]
预测值： [0.0800963] 真实值： [0.30635246]
预测值： [0.64058894] 真实值： [0.25307377]
预测值： [0.10725948] 真实值： [0.26536885]
预测值： [0.18544781] 真实值： [0.42827869]
预测值： [0.25549904] 真实值： [0.91803279]
预测值： [0.7068766] 真实值： [0.85143443]
预测值： [0.5907113] 真实值： [0.69262295]
预测值： [0.6265854] 真实值： [0.52561475]
预测值： [0.5343938] 真实值： [0.36372951]
预测值： [0.51204616] 真实值： [0.25922131]
预测值： [0.5417866] 真实值： [0.23053279]
预测值： [0.40929067] 真实值： [0.08913934]
预测值： [0.4795234] 真实值： [0.05430328]
预测值： [0.3282949] 真实值： [0.01844262]
预测值： [0.35781708] 真实值： [0.00512295]
预测值： [-0.2617036] 真实值： [0.00819672]
预测值： [0.19909549] 真实值： [0.03586066]
预测值： [-0.19555889] 真实值： [0.15983607]
预测值： [0.22322123] 真实值： [0.38729508]
预测值： [0.2611525] 真实值： [0.68340164]
预测值： [0.52377456] 真实值： [0.38627049]
预测值：

预测值： [0.10865649] 真实值： [0.0317623]
预测值： [-0.04927725] 真实值： [0.00922131]
预测值： [0.1851605] 真实值： [0.00512295]
预测值： [0.17621638] 真实值： [0.00717213]
预测值： [0.18868981] 真实值： [0.00819672]
预测值： [0.18594223] 真实值： [0.04405738]
预测值： [0.24334249] 真实值： [0.21516393]
预测值： [0.2818108] 真实值： [0.60963115]
预测值： [0.31442183] 真实值： [0.76741803]
预测值： [0.21026002] 真实值： [0.36577869]
预测值： [0.35001746] 真实值： [0.19979508]
预测值： [0.3180753] 真实值： [0.25204918]
预测值： [0.504564] 真实值： [0.3227459]
预测值： [0.47606048] 真实值： [0.33504098]
预测值： [0.54176164] 真实值： [0.2602459]
预测值： [0.571522] 真实值： [0.3329918]
预测值： [0.5896805] 真实值： [0.54815574]
预测值： [0.7527806] 真实值： [0.99282787]
预测值： [0.5350227] 真实值： [0.89754098]
预测值： [0.47730932] 真实值： [0.60963115]
预测值： [0.43386525] 真实值： [0.47131148]
预测值： [0.33895248] 真实值： [0.31659836]
预测值： [0.31327328] 真实值： [0.22028689]
预测值： [0.24198906] 真实值： [0.11065574]
预测值： [0.08312584] 真实值： [0.04918033]
预测值： [0.10960846] 真实值： [0.0204918]
预测值： [0.12970394] 真实值： [0.0102459]
预测值： [0.16732483] 真实值： [0.00614754]
预测值： [0

预测值： [0.38967845] 真实值： [0.0102459]
预测值： [0.35038355] 真实值： [0.00922131]
预测值： [-0.21889308] 真实值： [0.03893443]
预测值： [-0.28748295] 真实值： [0.1352459]
预测值： [0.22749034] 真实值： [0.39959016]
预测值： [0.80933595] 真实值： [0.75512295]
预测值： [0.12799838] 真实值： [0.36680328]
预测值： [0.39763537] 真实值： [0.18032787]
预测值： [0.6805265] 真实值： [0.2182377]
预测值： [0.74737257] 真实值： [0.29405738]
预测值： [0.75681543] 真实值： [0.30020492]
预测值： [0.8036817] 真实值： [0.27254098]
预测值： [0.8300856] 真实值： [0.33196721]
预测值： [0.81775457] 真实值： [0.50819672]
预测值： [0.7894164] 真实值： [0.92213115]
预测值： [0.6528223] 真实值： [0.90778689]
预测值： [0.6435402] 真实值： [0.54610656]
预测值： [0.59737676] 真实值： [0.45081967]
预测值： [0.4148426] 真实值： [0.32786885]
预测值： [0.6474239] 真实值： [0.23668033]
预测值： [0.5026022] 真实值： [0.15778689]
预测值： [0.9251092] 真实值： [0.10348361]
预测值： [0.8932589] 真实值： [0.04610656]
预测值： [0.8972436] 真实值： [0.0102459]
预测值： [0.90898466] 真实值： [0.01536885]
预测值： [0.90698105] 真实值： [0.0102459]
预测值： [0.9064271] 真实值： [0.04098361]
预测值： [0.90235406] 真实值： [0.13934426]
预测值： [0.

预测值： [0.5219567] 真实值： [0.33504098]
预测值： [0.59912366] 真实值： [0.31454918]
预测值： [0.6191682] 真实值： [0.35348361]
预测值： [0.65383464] 真实值： [0.45594262]
预测值： [0.59222955] 真实值： [0.96516393]
预测值： [0.5095163] 真实值： [0.85758197]
预测值： [0.25354445] 真实值： [0.54303279]
预测值： [0.13934006] 真实值： [0.44159836]
预测值： [0.01463569] 真实值： [0.19877049]
预测值： [-0.252432] 真实值： [0.18442623]
预测值： [-0.20813963] 真实值： [0.20286885]
预测值： [0.9631027] 真实值： [0.04918033]
预测值： [0.9559115] 真实值： [0.01639344]
预测值： [0.94775724] 真实值： [0.01536885]
预测值： [0.9539935] 真实值： [0.00614754]
预测值： [0.902951] 真实值： [0.00307377]
预测值： [0.49878988] 真实值： [0.04098361]
预测值： [0.91443306] 真实值： [0.18135246]
预测值： [0.8310721] 真实值： [0.47438525]
预测值： [0.4919385] 真实值： [0.83606557]
预测值： [0.4608203] 真实值： [0.39036885]
预测值： [0.5213648] 真实值： [0.23258197]
预测值： [0.36204472] 真实值： [0.23872951]
预测值： [0.36335796] 真实值： [0.33913934]
预测值： [0.36181155] 真实值： [0.31659836]
预测值： [0.26892534] 真实值： [0.27561475]
预测值： [0.38792992] 真实值： [0.30737705]
预测值： [0.5728038] 真实值： [0.47643443]
预测值： 

预测值： [-0.05026988] 真实值： [0.16393443]
预测值： [-0.01712681] 真实值： [0.17418033]
预测值： [-0.26383975] 真实值： [0.2397541]
预测值： [-0.28916532] 真实值： [0.23258197]
预测值： [-0.20616557] 真实值： [0.18545082]
预测值： [-0.12129626] 真实值： [0.23770492]
预测值： [-0.03901225] 真实值： [0.34631148]
预测值： [0.05893014] 真实值： [0.60553279]
预测值： [0.19959949] 真实值： [0.51844262]
预测值： [0.27071804] 真实值： [0.38114754]
预测值： [0.3184547] 真实值： [0.25]
预测值： [0.4488718] 真实值： [0.23770492]
预测值： [0.2493582] 真实值： [0.16393443]
预测值： [0.32046258] 真实值： [0.08811475]
预测值： [0.03327968] 真实值： [0.05532787]
预测值： [0.18911487] 真实值： [0.01946721]
预测值： [0.2548524] 真实值： [0.01127049]
预测值： [0.17542782] 真实值： [0.00819672]
预测值： [0.28563124] 真实值： [0.01331967]
预测值： [0.4643794] 真实值： [0.02663934]
预测值： [0.6802407] 真实值： [0.13319672]
预测值： [0.637964] 真实值： [0.34836066]
预测值： [0.501045] 真实值： [0.70901639]
预测值： [0.45019668] 真实值： [0.33401639]
预测值： [0.46742788] 真实值： [0.20081967]
预测值： [0.40557688] 真实值： [0.25614754]
预测值： [0.31503135] 真实值： [0.26741803]
预测值： [0.28281108] 真实值： [0.30532787]
预测

预测值： [-0.35165447] 真实值： [0.1147541]
预测值： [-0.38597563] 真实值： [0.07479508]
预测值： [-0.38167346] 真实值： [0.10963115]
预测值： [-0.34052333] 真实值： [0.06967213]
预测值： [0.25774994] 真实值： [0.05020492]
预测值： [0.21604005] 真实值： [0.01434426]
预测值： [-0.27283454] 真实值： [0.00409836]
预测值： [-0.347914] 真实值： [0.01229508]
预测值： [-0.37154123] 真实值： [0.02663934]
预测值： [0.24945918] 真实值： [0.06352459]
预测值： [0.26949456] 真实值： [0.14651639]
预测值： [0.2773416] 真实值： [0.19672131]
预测值： [0.22566904] 真实值： [0.26127049]
预测值： [0.21888202] 真实值： [0.40061475]
预测值： [0.22768444] 真实值： [0.45594262]
预测值： [0.26494205] 真实值： [0.5204918]
预测值： [0.21885636] 真实值： [0.4897541]
预测值： [0.13279086] 真实值： [0.49487705]
预测值： [0.17488153] 真实值： [0.45286885]
预测值： [0.15306035] 真实值： [0.32684426]
预测值： [0.323215] 真实值： [0.30327869]
预测值： [0.42253238] 真实值： [0.25409836]
预测值： [0.20005767] 真实值： [0.20184426]
预测值： [0.55325806] 真实值： [0.16086066]
预测值： [0.57056814] 真实值： [0.14959016]
预测值： [0.48638466] 真实值： [0.12807377]
预测值： [0.07417452] 真实值： [0.11885246]
预测值： [0.10522156] 真实值： [0.095

预测值： [0.13258429] 真实值： [0.36372951]
预测值： [0.03890866] 真实值： [0.31659836]
预测值： [0.05491833] 真实值： [0.34528689]
预测值： [0.04356133] 真实值： [0.40881148]
预测值： [0.03315906] 真实值： [0.24795082]
预测值： [0.19407295] 真实值： [0.10860656]
预测值： [-0.09526157] 真实值： [0.11065574]
预测值： [0.17531273] 真实值： [0.12397541]
预测值： [0.3692647] 真实值： [0.10758197]
预测值： [-0.07033607] 真实值： [0.09016393]
预测值： [0.06663404] 真实值： [0.03278689]
预测值： [-0.04429195] 真实值： [0.02766393]
预测值： [0.14234991] 真实值： [0.01434426]
预测值： [-0.09104073] 真实值： [0.00409836]
预测值： [-0.06662932] 真实值： [0.00204918]
预测值： [-0.09120737] 真实值： [0.00409836]
预测值： [0.0992327] 真实值： [0.02356557]
预测值： [0.11251949] 真实值： [0.10963115]
预测值： [0.12142967] 真实值： [0.32581967]
预测值： [0.28383666] 真实值： [0.60553279]
预测值： [0.17577304] 真实值： [0.28790984]
预测值： [0.2821169] 真实值： [0.13729508]
预测值： [0.29102564] 真实值： [0.17315574]
预测值： [0.23202659] 真实值： [0.23668033]
预测值： [0.32460353] 真实值： [0.21413934]
预测值： [0.30920765] 真实值： [0.18442623]
预测值： [0.28728828] 真实值： [0.21516393]
预测值： [-0.08670735] 真实值： [

In [22]:
final = sess.run(loss, feed_dict={x: x_test, y: y_test})
print(final)

0.116044104
